# Voxelization and Visualization of Arch Data

In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import IPython.display
import PIL.Image
import random
import enum

#### Visualization Configurations

In [ ]:
# change to True if you want to interact with the visualization windows
#interactive = not "CI" in os.environ
interactive = True
def jupyter_draw_geometries(
    geoms,
    window_name="Open3D",
    width=1920,
    height=1080,
    left=50,
    top=50,
    point_show_normal=False,
    mesh_show_wireframe=False,
    mesh_show_back_face=False,
    lookat=None,
    up=None,
    front=None,
    zoom=None,
):
    vis = o3d.visualization.Visualizer()
    vis.create_window(
        window_name=window_name,
        width=width,
        height=height,
        left=left,
        top=top,
        visible=True,  # If false, capture_screen_float_buffer() won't work.
    )
    vis.get_render_option().point_show_normal = point_show_normal
    vis.get_render_option().mesh_show_wireframe = mesh_show_wireframe
    vis.get_render_option().mesh_show_back_face = mesh_show_back_face
    for geom in geoms:
        vis.add_geometry(geom)
    if lookat is not None:
        vis.get_view_control().set_lookat(lookat)
    if up is not None:
        vis.get_view_control().set_up(up)
    if front is not None:
        vis.get_view_control().set_front(front)
    if zoom is not None:
        vis.get_view_control().set_zoom(zoom)
    if interactive:
        vis.run()
    else:
        for geom in geoms:
            vis.update_geometry(geom)
        vis.poll_events()
        vis.update_renderer()
    im = vis.capture_screen_float_buffer()
    vis.destroy_window()
    im = (255 * np.asarray(im)).astype(np.uint8)
    IPython.display.display(PIL.Image.fromarray(im, "RGB"))

o3d.visualization.draw_geometries = jupyter_draw_geometries

## Data Processing


### Helper Functions

In [ ]:
def load_point_clouds(dataset):
    pcds = []
    for data in dataset:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(data[:,:3])
        pcd.paint_uniform_color(get_random_color())
        pcds.append(pcd)
        print(len(data))
    return pcds

In [ ]:
def get_random_color():
    return [round(random.uniform(0.0, 1.0), 1), 
    round(random.uniform(0.0, 1.0), 1), 
    round(random.uniform(0.0, 1.0), 1)]

In [ ]:
def combine_point_clouds(pcds):
    pcd_combined = o3d.geometry.PointCloud()
    for point_id in range(len(pcds)):
        pcd_combined += pcds[point_id]
    return pcd_combined

### Load Data

In [ ]:
first_data = []
second_data = []

for i in range(1, 29):
    first_path = f'./test_data/arch/12_12_14_L4_V{i}.txt'
    first_data_arr = np.genfromtxt(first_path, delimiter=',')
    first_data.append(first_data_arr)
    
    if i in range (14, 17):
        continue
    
    second_path = f'./test_data/arch/13_11_23_L4_V{i}.txt'
    second_data_arr = np.genfromtxt(second_path, delimiter=',')
    second_data.append(second_data_arr)


In [ ]:
first_pcds = load_point_clouds(first_data)
second_pcds = load_point_clouds(second_data)

first_pcd_combined = combine_point_clouds(first_pcds)
second_pcd_combined = combine_point_clouds(second_pcds)

pcd_combined = combine_point_clouds([first_pcds_combined, second_pcds_combined])



### Write Data


In [ ]:
o3d.io.write_point_cloud("./out/first_combined_clouds.ply", first_pcd_combined)
o3d.io.write_point_cloud("./out/second_combined_clouds.ply", second_pcd_combined)
o3d.io.write_point_cloud("./out/combined_clouds.ply", pcd_combined)